# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
!pip install langchain_community

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [14]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.5)
# querying review so we can have some temp

In [16]:
prompt = ChatPromptTemplate.from_template(
    "Give a brief description about this {product}" 
)

In [17]:

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = df["Product"][0] #Queen Size Sheet Set
chain.run(product)

'This Queen Size Sheet Set includes a flat sheet, fitted sheet, and two pillowcases. The sheets are made of soft and durable material, providing a comfortable and cozy sleeping experience. The set is designed to fit a standard queen size bed and comes in a variety of colors and patterns to suit any bedroom decor. The sheets are easy to care for and maintain, making them a convenient and practical bedding option.'

## SimpleSequentialChain

In [19]:
from langchain.chains import SimpleSequentialChain

In [21]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Give a brief description about this {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [22]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Extract the most valuable information from {first_prompt}"    
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [23]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [24]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
This Queen Size Sheet Set includes a flat sheet, fitted sheet, and two pillowcases, all designed to fit a standard queen-size bed. Made of soft and high-quality material, this sheet set is comfortable to sleep on and easy to care for. The set typically comes in a variety of colors and patterns to suit different styles and preferences.
- Queen size sheet set includes flat sheet, fitted sheet, and two pillowcases
- Made of soft and high-quality material
- Comfortable to sleep on and easy to care for
- Available in a variety of colors and patterns to suit different styles and preferences

> Finished chain.


'- Queen size sheet set includes flat sheet, fitted sheet, and two pillowcases\n- Made of soft and high-quality material\n- Comfortable to sleep on and easy to care for\n- Available in a variety of colors and patterns to suit different styles and preferences'

**Repeat the above twice for different products**

In [79]:
product = df["Product"][1]
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
This Waterproof Phone Pouch is a durable and waterproof case designed to protect your phone from water, snow, dust, and dirt. It features a secure closure system to keep your phone safe and dry, while still allowing you to fully operate your phone's touchscreen and camera through the clear plastic window. The pouch also has an adjustable lanyard for convenient carrying and is compatible with most smartphones.
The most valuable information from this description is that the waterproof phone pouch is designed to protect your phone from water, snow, dust, and dirt. It features a secure closure system, allows you to fully operate your phone's touchscreen and camera, has an adjustable lanyard for convenient carrying, and is compatible with most smartphones.

> Finished chain.


"The most valuable information from this description is that the waterproof phone pouch is designed to protect your phone from water, snow, dust, and dirt. It features a secure closure system, allows you to fully operate your phone's touchscreen and camera, has an adjustable lanyard for convenient carrying, and is compatible with most smartphones."

In [78]:
product = df["Product"][2]
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
This luxury air mattress boasts a plush, comfortable surface that provides a restful night's sleep for its users. Made with high-quality materials and innovative design features, it offers customizable firmness levels and a durable construction that ensures long-lasting use. Perfect for indoor or outdoor use, this air mattress is a luxurious and convenient option for those looking to upgrade their sleeping experience.
- Plush and comfortable surface for a restful night's sleep
- High-quality materials and innovative design features
- Customizable firmness levels
- Durable construction for long-lasting use
- Suitable for indoor or outdoor use
- Luxurious and convenient option for upgrading sleeping experience

> Finished chain.


"- Plush and comfortable surface for a restful night's sleep\n- High-quality materials and innovative design features\n- Customizable firmness levels\n- Durable construction for long-lasting use\n- Suitable for indoor or outdoor use\n- Luxurious and convenient option for upgrading sleeping experience"

## SequentialChain

In [25]:
from langchain.chains import SequentialChain

In [86]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Identify The language of the review and if it is not in english then translate to english:[Original Language here] {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="english"
                    )


In [100]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review(shorter than original), mention in the beginning the original language like : [Original Language here] {english}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summarize"
                    )


In [111]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "translate to original language, you can find the original language at the start of the summary, if english was original already just skip this step: {summarize}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="original_summary"
                      )


In [112]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a short, friendly customer service response in both the original language and English. If English is the original language only one summary is sufficient. Use the following summaries as a basis for your response:\n\nOriginal Language Summary: {original_summary}\nEnglish Summary: {summarize}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="customer_response"
                     )


In [113]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english", "summarize", "original_summary", "customer_response"],
    verbose=True
)

In [114]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'english': "Language: French\n\nTranslation to English: I find the taste mediocre. The foam does not hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summarize': '[Original Language: French] The taste is mediocre, the foam does not hold. I usually buy the same ones in stores and the taste is better. Is this an old batch or counterfeit?',
 'original_summary': "Le goût est médiocre, la mousse ne tient pas. J'achète habituellement les mêmes dans les magasins et le goût est meilleur. Est-ce un vieux lot ou une contrefaçon?",
 'customer_response': "French Response: Bonjour, je suis désolé d'apprendre que vous n'êtes pas satisfait de notre produit. Pourriez-vous nous donner plus de détails sur votre achat, comme la date d'achat et le numéro de lot? Nous

**Repeat the above twice for different products or reviews**

In [115]:
review = df.Review[6]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'english': 'Language: Spanish\n\nTranslation: "It\'s a very pretty heater, it heats up very quickly, it is very functional, I just need to see how long it lasts, I have only been using it for 3 days."',
 'summarize': '[Original Language: Spanish]\n\nTranslation: "The heater is very attractive and efficient, heating up quickly. I have only been using it for 3 days, so I am curious to see how long it will last."',
 'original_summary': '"El calentador es muy atractivo y eficiente, se calienta rápidamente. Solo lo he estado usando durante 3 días, así que tengo curiosidad por ver cuánto durará."',
 'customer_response': "¡Gracias por tu comentario! Nos alegra que estés disfrutando de tu nuevo calentador. Estamos seguros de que te seguirá manteniendo caliente y cómodo durante mucho tiempo. ¡Gracias por elegir nuestra marca! \n\nThank you for your feedback! We'r

In [116]:
review = df.Review[4]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'english': 'The language of the review is English. No translation is needed.',
 'summarize': '[Original Language: English]  This review praises the product for its high quality, durability, and functionality. The reviewer mentions that they have been using it for a while and it still looks new. They also appreciate the fast shipping and excellent customer service. Overall, they highly recommend the product.',
 'original_summary': '[Original Language: English] This review praises the product for its high quality, durability, and functionality. The reviewer men

In [118]:
# code kind of works but the next step is probably helping with the awkwardness

## Router Chain

In [119]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [120]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [121]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [122]:
llm = ChatOpenAI(temperature=0)

In [123]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [124]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [125]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [126]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [127]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\nicol\AppData\Local\Temp\ipykernel_1924\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [128]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [129]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [130]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This genetic information is crucial for maintaining the integrity and stability of our cells and ultimately our entire body. \n\nAdditionally, DNA is constantly being replicated and repaired in our cells to ensure that genetic information is passed on accurately to the next generation of cells. This process is essential for the growth, development, and overall health of our bodies. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and maintenance of a

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [131]:
chain.run("Who fought in the thirty year war again?")



> Entering new MultiPromptChain chain...
History: {'input': 'Who fought in the thirty year war again?'}
> Finished chain.


"The Thirty Years' War was fought primarily in Central Europe from 1618 to 1648. The main belligerents were the Catholic Habsburg Monarchy, led by the Holy Roman Emperor Ferdinand II, and various Protestant states, including the Kingdom of Sweden, the Dutch Republic, and several German states. The war also involved other European powers, such as France and Spain, who supported different sides at various points during the conflict."

In [133]:
chain.run("is it chain.run() or chain.invoke() for a LangChain?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'is it chain.run() or chain.invoke() for a LangChain?'}
> Finished chain.


'It is chain.run() for a LangChain. The run() method is typically used to execute the chain of commands or functions in a LangChain object. The invoke() method is not commonly used in this context.'